In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from pathlib import Path
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
#from keras.layers import Dense, GlobalAveragePooling2D, Input
#from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
#from keras.applications import efficientnet_v2

In [ ]:
path = Path('../input/sorghum-id-fgvc-9')

In [ ]:
train_df = pd.read_csv(path/'train_cultivar_mapping.csv')

In [ ]:
train_df.count()

In [ ]:
actual_images = [img for img in os.listdir(path/'train_images') if img in train_df.image.to_list()]

In [ ]:
len(actual_images)

In [ ]:
train_df = train_df[train_df.image.isin(actual_images)]

In [ ]:
train_df.shape

In [ ]:
train_df.head()

In [ ]:
model = keras.models.load_model('../input/sorghum-keras-efficientnetv2b3-model/Sorghum_keras_best_model.h5')

In [ ]:
IMG_SIZE = 256
BATCH_SIZE = 32
EPOCHS = 30

In [ ]:
test_filenames = os.listdir(path/'test')
test_df = pd.DataFrame({'filename' : test_filenames})

test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_dataframe(
    dataframe = test_df,
    directory = path/'test',
    x_col = 'filename',
    y_col = None,
    class_mode = None,
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    shuffle = False
)

In [ ]:
classes = list(train_df.cultivar.unique())
len(classes)

In [ ]:
preds = model.predict(test_generator)
pred_idx = np.argmax(preds, axis=1)
predictions = [classes[i] for i in pred_idx]

In [ ]:
test_df['cultivar'] = predictions

In [ ]:
test_df.to_csv('submission.csv', index=False)